In [1]:
import os
import shutil
import requests
from requests.exceptions import RequestException

# Define the base path and folder names
base_path = r"D:\wav2lip-HD-main\Wav2Lip-GFPGAN"
wav2lip_path = os.path.join(base_path, 'Wav2Lip-master')
gfpgan_path = os.path.join(base_path, 'GFPGAN-master')

# Ensure the necessary directories exist
os.makedirs(os.path.join(wav2lip_path, 'face_detection/detection/sfd'), exist_ok=True)
os.makedirs(os.path.join(wav2lip_path, 'checkpoints'), exist_ok=True)
os.makedirs(os.path.join(gfpgan_path, 'experiments/pretrained_models'), exist_ok=True)

# Function to download a file from a URL with retries
def download_file(url, dest_path, retries=3):
    for attempt in range(retries):
        try:
            response = requests.get(url, stream=True, timeout=60)
            response.raise_for_status()  # Check for errors
            with open(dest_path, 'wb') as file:
                for chunk in response.iter_content(chunk_size=8192):
                    if chunk:  # filter out keep-alive new chunks
                        file.write(chunk)
            print(f"Downloaded successfully: {dest_path}")
            return
        except RequestException as e:
            print(f"Error downloading {url}: {e}. Retrying {attempt + 1}/{retries}")
        except IncompleteRead as e:
            print(f"Incomplete read error: {e}. Retrying {attempt + 1}/{retries}")
    print(f"Failed to download {url} after {retries} attempts")

# Paths to download files
s3fd_path = os.path.join(wav2lip_path, 'face_detection/detection/sfd/s3fd.pth')
wav2lip_gan_path = os.path.join(wav2lip_path, 'checkpoints/wav2lip_gan.pth')
gfpgan_pth_path = os.path.join(gfpgan_path, 'experiments/pretrained_models/GFPGANv1.3.pth')

# Download the necessary files with retry logic
download_file('https://www.adrianbulat.com/downloads/python-fan/s3fd-619a316812.pth', s3fd_path)
download_file('https://iiitaphyd-my.sharepoint.com/personal/radrabha_m_research_iiit_ac_in/_layouts/15/download.aspx?share=EdjI7bZlgApMqsVoEUUXpLsBxqXbn5z8VTmoxp55YNDcIA', wav2lip_gan_path)
download_file('https://github.com/TencentARC/GFPGAN/releases/download/v1.3.0/GFPGANv1.3.pth', gfpgan_pth_path)


Downloaded successfully: D:\wav2lip-HD-main\Wav2Lip-GFPGAN\Wav2Lip-master\face_detection/detection/sfd/s3fd.pth
Downloaded successfully: D:\wav2lip-HD-main\Wav2Lip-GFPGAN\Wav2Lip-master\checkpoints/wav2lip_gan.pth
Downloaded successfully: D:\wav2lip-HD-main\Wav2Lip-GFPGAN\GFPGAN-master\experiments/pretrained_models/GFPGANv1.3.pth


In [2]:
# Install distutils using ensurepip
import ensurepip
ensurepip.bootstrap()

# Alternatively, install setuptools which includes distutils
%pip install setuptools

# Install the required Python packages
%pip install -r requirements.txt
%pip install -U librosa==0.8.1


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
  Using cached librosa-0.10.0-py3-none-any.whl.metadata (8.3 kB)
  Using cached numpy-1.24.0.tar.gz (10.9 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
Note: you may need to restart the kernel to use updated packages.


ERROR: Exception:
Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.12_3.12.1008.0_x64__qbz5n2kfra8p0\Lib\site-packages\pip\_internal\cli\base_command.py", line 180, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.12_3.12.1008.0_x64__qbz5n2kfra8p0\Lib\site-packages\pip\_internal\cli\req_command.py", line 245, in wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.12_3.12.1008.0_x64__qbz5n2kfra8p0\Lib\site-packages\pip\_internal\commands\install.py", line 377, in run
    requirement_set = resolver.resolve(
                      ^^^^^^^^^^^^^^^^^
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.12_3.12.1008.0_x64__qbz5n2kfra8p0\Lib\site-packages\pip\_internal\resolution\resolvelib\resolver.py", line 95, in res

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
import os

base_path = r"D:\wav2lip-HD-main\Wav2Lip-GFPGAN\Wav2Lip-master"

for root, dirs, files in os.walk(base_path):
    for file in files:
        if file.endswith(".py"):
            file_path = os.path.join(root, file)
            with open(file_path, 'r') as f:
                content = f.read()
            
            updated_content = content.replace('np.float', 'float')

            with open(file_path, 'w') as f:
                f.write(updated_content)

print("Replacement complete.")


Replacement complete.


In [4]:
import os
import subprocess

base_path = r"D:\wav2lip-HD-main\Wav2Lip-GFPGAN"
outputPath = os.path.join(base_path, 'outputs')
inputAudioPath = os.path.join(base_path, 'inputs', 'kimk_audio.mp3')
inputVideoPath = os.path.join(base_path, 'inputs', 'kim_7s_raw.mp4')
lipSyncedOutputPath = os.path.join(outputPath, 'result.mp4')
tempPath = os.path.join(base_path, 'temp')

if not os.path.exists(outputPath):
    os.makedirs(outputPath)
if not os.path.exists(tempPath):
    os.makedirs(tempPath)

wav2lipFolderName = 'Wav2Lip-master'
os.chdir(os.path.join(base_path, wav2lipFolderName))

# Ensure FFmpeg is correctly added to the PATH
os.environ["PATH"] += os.pathsep + r"C:\Path\To\FFmpeg\bin"

# Command to run the inference script
command = [
    "python", "inference.py",
    "--checkpoint_path", "checkpoints/wav2lip.pth",
    "--face", inputVideoPath,
    "--audio", inputAudioPath,
    "--outfile", lipSyncedOutputPath
]

try:
    result = subprocess.run(command, capture_output=True, text=True, check=True)
    print("Subprocess output:\n", result.stdout)
    print("Subprocess error (if any):\n", result.stderr)
except subprocess.CalledProcessError as e:
    print("Error occurred:")
    print("Return code:", e.returncode)
    print("Command:", e.cmd)
    print("Output:\n", e.output)
    print("Error:\n", e.stderr)

# Running the ffmpeg command manually for debugging
ffmpeg_command = [
    "ffmpeg", "-i", inputAudioPath, "-f", "wav", os.path.join(tempPath, "temp_audio.wav")
]

try:
    ffmpeg_result = subprocess.run(ffmpeg_command, capture_output=True, text=True, check=True)
    print("FFmpeg command output:\n", ffmpeg_result.stdout)
    print("FFmpeg command error (if any):\n", ffmpeg_result.stderr)
except subprocess.CalledProcessError as e:
    print("Error occurred during FFmpeg execution:")
    print("Return code:", e.returncode)
    print("Command:", e.cmd)
    print("Output:\n", e.output)
    print("Error:\n", e.stderr)



Error occurred:
Return code: 1
Command: ['python', 'inference.py', '--checkpoint_path', 'checkpoints/wav2lip.pth', '--face', 'D:\\wav2lip-HD-main\\Wav2Lip-GFPGAN\\inputs\\kim_7s_raw.mp4', '--audio', 'D:\\wav2lip-HD-main\\Wav2Lip-GFPGAN\\inputs\\kimk_audio.mp3', '--outfile', 'D:\\wav2lip-HD-main\\Wav2Lip-GFPGAN\\outputs\\result.mp4']
Output:
 Using cpu for inference.
Reading video frames...
Number of frames available for inference: 223
Extracting raw audio...

Error:
 ffmpeg version N-115838-g4e4444f97c-20240615 Copyright (c) 2000-2024 the FFmpeg developers
  built with gcc 13.2.0 (crosstool-NG 1.26.0.65_ecc5e41)
  configuration: --prefix=/ffbuild/prefix --pkg-config-flags=--static --pkg-config=pkg-config --cross-prefix=x86_64-w64-mingw32- --arch=x86_64 --target-os=mingw32 --enable-gpl --enable-version3 --disable-debug --disable-w32threads --enable-pthreads --enable-iconv --enable-libxml2 --enable-zlib --enable-libfreetype --enable-libfribidi --enable-gmp --enable-fontconfig --enable-li

In [ ]:
import urllib.request

url = "https://github.com/TencentARC/GFPGAN/releases/download/v1.3.0/GFPGANv1.3.pth"
save_path = f"{r"D:\wav2lip-HD-main\Wav2Lip-GFPGAN\GFPGAN-master"}/experiments/pretrained_models/GFPGANv1.3.pth"

urllib.request.urlretrieve(url, save_path)


('D:\\wav2lip-HD-main\\Wav2Lip-GFPGAN\\GFPGAN-master/experiments/pretrained_models/GFPGANv1.3.pth',
 <http.client.HTTPMessage at 0x2a75c0e09e0>)

In [ ]:
import cv2
from tqdm import tqdm
from os import path

import os

inputVideoPath = r'D:\wav2lip-HD-main\Wav2Lip-GFPGAN\inputs\kim_7s_raw.mp4'
unProcessedFramesFolderPath = r'D:\wav2lip-HD-main\Wav2Lip-GFPGAN\outputs\frames'

if not os.path.exists(unProcessedFramesFolderPath):
  os.makedirs(unProcessedFramesFolderPath)

vidcap = cv2.VideoCapture(inputVideoPath)
numberOfFrames = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
fps = vidcap.get(cv2.CAP_PROP_FPS)
print("FPS: ", fps, "Frames: ", numberOfFrames)

for frameNumber in tqdm(range(numberOfFrames)):
    _,image = vidcap.read()
    cv2.imwrite(path.join(unProcessedFramesFolderPath, str(frameNumber).zfill(4)+'.jpg'), image)


  0%|          | 0/223 [00:00<?, ?it/s]

FPS:  30.0 Frames:  223


100%|██████████| 223/223 [00:08<00:00, 26.45it/s]


In [3]:
from moviepy.editor import VideoFileClip, concatenate_videoclips, AudioFileClip
import os

# Directory containing frames
frames_dir = r'D:\wav2lip-HD-main\Wav2Lip-GFPGAN\outputs\frames'

# Load audio file
audio_path = r'D:\wav2lip-HD-main\Wav2Lip-GFPGAN\inputs\kimk_audio.mp3'
audio_clip = AudioFileClip(audio_path)

# Get list of frames (adjust file extension as per your frames)
frame_files = [os.path.join(frames_dir, f) for f in os.listdir(frames_dir) if f.endswith('.jpg')]
frame_files.sort()  # Sort frames to ensure they are in order

# Check if frames exist
if not frame_files:
    print(f"No JPEG frames found in {frames_dir}")
    exit()

# Create list of VideoFileClip objects from frames
video_clips = [VideoFileClip(frame) for frame in frame_files]

# Concatenate video clips into a single clip
video_clip = concatenate_videoclips(video_clips)

# Set audio for the video clip
video_clip = video_clip.set_audio(audio_clip)

# Write synchronized video with audio
output_path = r'D:\wav2lip-HD-main\Wav2Lip-GFPGAN\outputs\synchronizedd_video.mp4'
video_clip.write_videofile(output_path, codec='libx264', audio_codec='aac')

# Close the video and audio clips
video_clip.close()
audio_clip.close()

print("Synchronized video with audio saved:", output_path)


chunk:  17%|█▋        | 30/180 [00:00<00:00, 297.53it/s, now=None]

Moviepy - Building video D:\wav2lip-HD-main\Wav2Lip-GFPGAN\outputs\synchronizedd_video.mp4.
MoviePy - Writing audio in synchronizedd_videoTEMP_MPY_wvf_snd.mp4


t:   1%|          | 2/223 [00:00<00:11, 18.69it/s, now=None]       

MoviePy - Done.
Moviepy - Writing video D:\wav2lip-HD-main\Wav2Lip-GFPGAN\outputs\synchronizedd_video.mp4



Moviepy - Done !
Moviepy - video ready D:\wav2lip-HD-main\Wav2Lip-GFPGAN\outputs\synchronizedd_video.mp4
Synchronized video with audio saved: D:\wav2lip-HD-main\Wav2Lip-GFPGAN\outputs\synchronizedd_video.mp4


In [9]:
import os
from subprocess import run, CalledProcessError

# Define paths
wav2lip_repo_path = r'D:\wav2lip-HD-main\Wav2Lip-GFPGAN\GFPGAN-master'
input_video_path = r'D:\wav2lip-HD-main\Wav2Lip-GFPGAN\inputs\kim_7s_raw.mp4'
input_audio_path = r'D:\wav2lip-HD-main\Wav2Lip-GFPGAN\inputs\kim_audio.mp3'
output_video_path = r'D:\wav2lip-HD-main\Wav2Lip-GFPGAN\outputs\result_video.mp4'
inference_script_path = os.path.join(wav2lip_repo_path, 'inference_gfpgan.py')

# Ensure the paths are correct
assert os.path.exists(inference_script_path), f"{inference_script_path} does not exist"
assert os.path.exists(input_video_path), f"{input_video_path} does not exist"
assert os.path.exists(input_audio_path), f"{input_audio_path} does not exist"

# Run Wav2Lip inference
command = [
    'python', inference_script_path,
    '--checkpoint_path', os.path.join(wav2lip_repo_path, 'checkpoints/wav2lip_gan.pth'),
    '--face', input_video_path,
    '--audio', input_audio_path,
    '--outfile', output_video_path
]

try:
    result = run(command, capture_output=True, text=True, check=True)
    print(result.stdout)
    print(f"Lip-synced video saved at {output_video_path}")
except CalledProcessError as e:
    print("Error:", e.stderr)


Error: Traceback (most recent call last):
  File "D:\wav2lip-HD-main\Wav2Lip-GFPGAN\GFPGAN-master\inference_gfpgan.py", line 7, in <module>
    from basicsr.utils import imwrite
ModuleNotFoundError: No module named 'basicsr'



In [10]:
import subprocess

input_video_path = r'D:\wav2lip-HD-main\Wav2Lip-GFPGAN\outputs\inppp.mp4'
input_audio_path = r'D:\wav2lip-HD-main\Wav2Lip-GFPGAN\inputs\kimk_audio.mp3'
output_final_video_path = r'D:\wav2lip-HD-main\Wav2Lip-GFPGAN\outputs\final_video_with_audio.mp4'

command = [
    'ffmpeg', '-y', '-i', input_video_path, '-i', input_audio_path,
    '-c:v', 'copy', '-c:a', 'aac', '-strict', 'experimental', '-map', '0:v:0', '-map', '1:a:0',
    '-shortest', output_final_video_path
]

subprocess.run(command, check=True)
print(f"Final video with audio saved at {output_final_video_path}")


Final video with audio saved at D:\wav2lip-HD-main\Wav2Lip-GFPGAN\outputs\final_video_with_audio.mp4
